# Working with Schedulers

In non-deterministic models the notion of a scheduler (or policy) is important.
The scheduler determines which action to take at each state.

For a given reachability property, Storm can return the scheduler realizing the resulting probability.

## Examining Schedulers for MDPs

[01-schedulers.py](https://github.com/moves-rwth/stormpy/blob/master/examples/schedulers/01-schedulers.py)

As in [Getting Started](../getting_started.ipynb), we import some required modules and build a model from the example files:

In [1]:
>>> import stormpy
>>> import stormpy.examples
>>> import stormpy.examples.files

>>> path = stormpy.examples.files.prism_mdp_coin_2_2
>>> formula_str = "Pmin=? [F \"finished\" & \"all_coins_equal_1\"]"
>>> program = stormpy.parse_prism_program(path)
>>> formulas = stormpy.parse_properties(formula_str, program)
>>> model = stormpy.build_model(program, formulas)

Next we check the model and make sure to extract the scheduler:

In [2]:
>>> result = stormpy.model_checking(model, formulas[0], extract_scheduler=True)

The result then contains the scheduler we want:

In [3]:
>>> assert result.has_scheduler
>>> scheduler = result.scheduler
>>> assert scheduler.memoryless
>>> assert scheduler.deterministic
>>> print(scheduler)

___________________________________________________________________
Fully defined memoryless deterministic scheduler:
model state:    choice(s)
           0    0
           1    0
           2    1
           3    0
           4    0
           5    0
           6    0
           7    0
           8    0
           9    1
          10    0
          11    0
          12    0
          13    0
          14    0
          15    0
          16    0
          17    0
          18    0
          19    0
          20    0
          21    1
          22    1
          23    0
          24    1
          25    0
          26    0
          27    0
          28    0
          29    0
          30    1
          31    0
          32    0
          33    0
          34    0
          35    0
          36    0
          37    1
          38    1
          39    0
          40    0
          41    0
          42    0
          43    0
          44    0
          45    1
          46    0
          

To get the information which action the scheduler chooses in which state, we can simply iterate over the states:

In [4]:
>>> for state in model.states:
...     choice = scheduler.get_choice(state)
...     action = choice.get_deterministic_choice()
...     print("In state {} choose action {}".format(state, action))

In state 0 choose action 0
In state 1 choose action 0
In state 2 choose action 1
In state 3 choose action 0
In state 4 choose action 0
In state 5 choose action 0
In state 6 choose action 0
In state 7 choose action 0
In state 8 choose action 0
In state 9 choose action 1
In state 10 choose action 0
In state 11 choose action 0
In state 12 choose action 0
In state 13 choose action 0
In state 14 choose action 0
In state 15 choose action 0
In state 16 choose action 0
In state 17 choose action 0
In state 18 choose action 0
In state 19 choose action 0
In state 20 choose action 0
In state 21 choose action 1
In state 22 choose action 1
In state 23 choose action 0
In state 24 choose action 1
In state 25 choose action 0
In state 26 choose action 0
In state 27 choose action 0
In state 28 choose action 0
In state 29 choose action 0
In state 30 choose action 1
In state 31 choose action 0
In state 32 choose action 0
In state 33 choose action 0
In state 34 choose action 0
In state 35 choose action 0
In

## Examining Schedulers for Markov automata

[02-schedulers.py](https://github.com/moves-rwth/stormpy/blob/master/examples/schedulers/02-schedulers.py)

Currently there is no support yet for scheduler extraction on MAs.
However, if the timing information is not relevant for the property, we can circumvent this lack by first transforming the MA to an MDP.

We build the model as before:

In [5]:
>>> path = stormpy.examples.files.prism_ma_simple
>>> formula_str = "Tmin=? [ F s=4 ]"

>>> program = stormpy.parse_prism_program(path, False, True)
>>> formulas = stormpy.parse_properties_for_prism_program(formula_str, program)
>>> ma = stormpy.build_model(program, formulas)

Next we transform the continuous-time model into a discrete-time model.
Note that all timing information is lost at this point:

In [6]:
>>> mdp, mdp_formulas = stormpy.transform_to_discrete_time_model(ma, formulas)
>>> assert mdp.model_type == stormpy.ModelType.MDP

After the transformation we have obtained an MDP where we can extract the scheduler as shown before:

In [7]:
>>> result = stormpy.model_checking(mdp, mdp_formulas[0], extract_scheduler=True)
>>> scheduler = result.scheduler
>>> print(scheduler)


___________________________________________________________________
Fully defined memoryless deterministic scheduler:
model state:    choice(s)
           0    1
           1    0
           2    0
           3    0
           4    0
___________________________________________________________________

